In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds
from tensorflow import keras

In [2]:
root_logdir = os.path.join(os.curdir, "my_tb_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d_%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [3]:
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

In [4]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)

In [5]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [6]:
batch_size = 8
train_set = train_set_raw.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [7]:
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [8]:
for layer in base_model.layers:
    layer.trainable = False
optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [9]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [10]:
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5
344/344 [==============================] - 31s 77ms/step - loss: 3.0860 - accuracy: 0.8089 - val_loss: 1.9439 - val_accuracy: 0.8657
Epoch 2/5
344/344 [==============================] - 26s 75ms/step - loss: 0.7133 - accuracy: 0.9248 - val_loss: 1.8550 - val_accuracy: 0.8566
Epoch 3/5
344/344 [==============================] - 26s 75ms/step - loss: 0.3337 - accuracy: 0.9488 - val_loss: 1.8578 - val_accuracy: 0.8457
Epoch 4/5
344/344 [==============================] - 26s 76ms/step - loss: 0.2154 - accuracy: 0.9611 - val_loss: 1.8066 - val_accuracy: 0.8439
Epoch 5/5
344/344 [==============================] - 26s 76ms/step - loss: 0.1342 - accuracy: 0.9691 - val_loss: 1.6338 - val_accuracy: 0.8494


In [11]:
tf.config.experimental.get_memory_info('GPU:0')

{'current': 94383616, 'peak': 1883904768}

In [12]:
for layer in base_model.layers:
    layer.trainable=True
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                 nesterov=True, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set, epochs=40, validation_data=valid_set, callbacks=[tensorboard_cb])

Epoch 1/40
344/344 [==============================] - 108s 299ms/step - loss: 1.0578 - accuracy: 0.6134 - val_loss: 8.2194 - val_accuracy: 0.4374
Epoch 2/40
344/344 [==============================] - 103s 299ms/step - loss: 0.5697 - accuracy: 0.7983 - val_loss: 0.5508 - val_accuracy: 0.7967
Epoch 3/40
344/344 [==============================] - 103s 300ms/step - loss: 0.3841 - accuracy: 0.8583 - val_loss: 0.3222 - val_accuracy: 0.8784
Epoch 4/40
344/344 [==============================] - 103s 300ms/step - loss: 0.2344 - accuracy: 0.9168 - val_loss: 0.3508 - val_accuracy: 0.9093
Epoch 5/40
344/344 [==============================] - 103s 301ms/step - loss: 0.1389 - accuracy: 0.9528 - val_loss: 0.5531 - val_accuracy: 0.8802
Epoch 6/40
344/344 [==============================] - 104s 302ms/step - loss: 0.1151 - accuracy: 0.9644 - val_loss: 0.3086 - val_accuracy: 0.9056
Epoch 7/40
344/344 [==============================] - 104s 302ms/step - loss: 0.0818 - accuracy: 0.9717 - val_loss: 0.8749 -

In [13]:
model.evaluate(test_set)

46/46 [==============================] - 3s 62ms/step - loss: 0.6338 - accuracy: 0.9155


[0.6337935924530029, 0.9155313372612]

In [14]:
tf.config.experimental.get_memory_info('GPU:0')

{'current': 177932544, 'peak': 1883904768}